## Trabajo práctico 4 - Balance

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

Trabajando sobre la imagen sombreado:
1. Encontrar el umbral con búsqueda binaria según lo indicado en el Jupyter Notebook y los videos. Comparar el resultado con la binarización por el método de Otzu.
2. Programar el método de binarización local por Bernsen. Ajustarlo para obtener buenos resultados de binarización sobre esta imagen.

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.show()  

In [ ]:
img = cv.imread('Sombreado.png', cv.IMREAD_GRAYSCALE)
plotter(img, 'Original Image', grayScale=True)

## 1) Búsqueda del umbral

In [ ]:
#Ejercicio: Completar lo que falte y comparar contra el método de Otsu (por ejemplo restando las salidas)

#Paso1: Definir umbral inicial (en general la media de la imagen)

def buscar_umbral(img, umbral=128, delta_T=1.0):
    
    #Paso2: Dividir la imagen en dos partes
    # Usar np.where para encontrar los índices
    xp1,yp1 = np.where(img < umbral)
    xp2,yp2 = np.where(img >= umbral)
    

    #Paso3: Encontrar la media de cada parte
    media_p1 = np.mean(img[xp1][yp1])
    media_p2 = np.mean(img[xp2][yp2])

    #Paso4: Calcular el nuevo umbral (promedio entre media anterior y actual)
    nuevo_umbral = (media_p1 + media_p2) / 2
    
    #Paso5: Criterio de detención (o recalculo)
    if abs(nuevo_umbral - umbral) < delta_T:
        return nuevo_umbral
    else:
        return buscar_umbral(img, umbral=nuevo_umbral)

In [ ]:
umbral = np.round(buscar_umbral(img, round(np.mean(img))))

In [ ]:
# Funcion de binarización (a mano)
def global_threshold(image, thres_value, val_high, val_low):
    img = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if image[i,j] > thres_value:
                img[i,j] = val_high
            else:
                img[i,j] = val_low
    return img


# Nueva figura
fig = plt.figure(figsize=(16,4))

# Imagen binarizada (a mano - Fijo)
ax2=plt.subplot(221)
img_bin = global_threshold(img, umbral, 1, 0);
ax2.imshow(img_bin,cmap='gray', vmin=0, vmax=1)
ax2.set_title('Binarizada a mano')

# Imagen binarizada (embebida - Fijo) - Parámetros: Imag_original,umbral,valor_máximo,método
ax3=plt.subplot(222)
ret, thresh = cv.threshold(img,umbral,255,cv.THRESH_BINARY)
ax3.imshow(thresh,cmap='gray', vmin=0, vmax=1)
ax3.set_title('Binarizada OpenCV, Fijo')

# Imagen binarizada (embebida - Otsu) - Parámetros: Imag_original,umbral,valor_máximo,método
ax4=plt.subplot(223)
ret, thresh = cv.threshold(img,umbral,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
ax4.imshow(thresh,cmap='gray', vmin=0, vmax=1)
ax4.set_title('Binarizada OpenCV, Otsu')

# Imagen binarizada (embebida - Mediana) - Parámetros: Imag_original,valor_máximo,método adaptativo,tipo de umbral,tamaño de bloque,C
ax4=plt.subplot(224)
thresh = cv.adaptiveThreshold(img,255,cv.ADAPTIVE_THRESH_MEAN_C,cv.THRESH_BINARY,11,2)
ax4.imshow(thresh,cmap='gray', vmin=0, vmax=1)
ax4.set_title('Binarizada OpenCV, Mediana')

plt.show()

## 2) Método de binarización local por Bernsen

In [ ]:
# Implementar para una ventana de 3x3, 5x5 o 7x7.
# Contraste: Máximo-Mínimo  de la ventanida, bah contraste_local es el de la ventanita
# gris_medio: Media del bloquecito de 3x3, etc

#contraste_referencia = se tiene que ingresar esto
#if(contraste_local < contraste_referencia)
#  pixel = (gris_medio >= 128)? 255:0
#else
#  pixel = (pixel >= gris_medio)? 255:0

In [ ]:
def getSubmatrix(matrix, row, col, size):
    submatrix = []
    for i in range(row, (row + size) if row + size < len(matrix) else len(matrix)):
        for j in range(col, (col + size) if col + size < len(matrix[0]) else len(matrix[0])):
            submatrix.append(matrix[i][j])
    return submatrix

In [ ]:
def bernsen(image, windowSize, contrast):
    aux = []
    for row in range(0, len(image), windowSize):
        for col in range(0, len(image[0]), windowSize):
            submatrix = getSubmatrix(image, row, col, windowSize) #Generamos la ventana de tamanio windowSize
            midGray = np.mean(submatrix) 
            maxIntensity, minIntensity = max(submatrix), min(submatrix)
            localContrast = maxIntensity - minIntensity
            aux.append(localContrast)
            
            for i in range(len(submatrix)):
                if (localContrast < contrast):
                    submatrix[i] = 255 if midGray >= 128 else 0
                else:
                    submatrix[i] = 255 if submatrix[i] >= midGray else 0
            
            #Ahora actualizo los valores de la imagen original
            auxCont = 0
            for i in range(row, row + windowSize):
                if (i >= len(image)):
                    break
                for j in range(col,  col + windowSize):
                    if (j >= len(image[0])):
                        break
                    image[i][j] = submatrix[auxCont]
                    auxCont += 1
            
    print("Min local contrast = {} // Max local contrast = {}".format(min(aux), max(aux)))

In [ ]:
contrasts = range(1,11) 
for contrast in contrasts:
    print('Contraste {}'.format(contrast))
    for windowSize in [3,5,7]:
        imgCopy = img.copy()
        bernsen(imgCopy, windowSize, contrast)
        title = 'Bernsen {}x{}'.format(windowSize, windowSize)
        plotter(imgCopy, title, grayScale=True)